# Estandarización de los datos numericos. 

```
Revisar los imports que no se utilicen al final del todo.
```

In [1]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Vigilar progreso bucles
# -----------------------------------------------------------------------
from tqdm import tqdm

# Gestionar los warnings
# -----------------------------------------------------------------------
import warnings

# modificar el path
# -----------------------------------------------------------------------
import sys
sys.path.append("../..")

# importar funciones de soporte
# -----------------------------------------------------------------------
from src import soporte_preprocesamiento as sup_prep
from src import soporte_nulos as sup_nul


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler

##aplicar configuraciones
#------------------------------------------------------------------------
warnings.filterwarnings('ignore')
pd.set_option('display.max_info_columns', 50)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format) #eliminamos la notacion cientifica

tqdm.pandas()

---
---

## Importamos los datos ya tratados en el notebook ```1_EDA.ipynb```

In [2]:
df = pd.read_csv('../../datos/datos_modelo1/tratados/nonuls_propiedades_provincia_madrid.csv', index_col=0).reset_index(drop=True)
df.sample()

,numPhotos,price,propertyType,size,exterior,rooms,bathrooms,municipality,showAddress,distance,hasVideo,status,newDevelopment,hasPlan,has3DTour,has360,topNewDevelopment,floor,district,neighborhood,hasLift
99,17,720.00,studio,45.00,True,0,1,San Sebastián de los Reyes,True,15433,True,good,False,True,False,True,False,5,Vega de la Moraleja,Desconocido,True


---
---

## Realizamos la estandarización

### Dado que tenemos pocos outlayers y estos son reales (por lo que quiero darles importancia) *voy a utilizar el metodo de RobustScaler*.

In [3]:
df_transformar = df.copy()
X = df_transformar.drop(columns=['price']) # excluimos nuestra variable respuesta
df_transformar_num = X.select_dtypes(np.number)
df_transformar_num.sample()

,numPhotos,size,rooms,bathrooms,distance
244,33,62.00,2,1,26890


In [4]:
robust_scaler = RobustScaler()
columnas_numericas = df_transformar_num.columns
df_transf_robust = pd.DataFrame(robust_scaler.fit_transform(df_transformar_num), columns=columnas_numericas)

### Sustituimos los valores numericos ya transformados.

In [5]:
sup_nul.sustituir_valores_df(df_transformar, df_transf_robust, columnas=columnas_numericas)

In [6]:
df_transformar.head(2)

,numPhotos,price,propertyType,size,exterior,rooms,bathrooms,municipality,showAddress,distance,hasVideo,status,newDevelopment,hasPlan,has3DTour,has360,topNewDevelopment,floor,district,neighborhood,hasLift
0,-0.62,750.00,flat,0.12,True,-1.00,0.00,Madrid,True,-0.25,False,good,False,False,False,False,False,3,Hortaleza,Canillas,True
1,0.08,750.00,flat,0.52,True,0.00,0.00,San Sebastián de los Reyes,False,0.18,True,good,False,True,False,False,False,bj,Centro Urbano,Desconocido,False


<h3 style="color: #9966CC">
(meter comparacion visual para ver la dispersion de los datos antes y ahora)
</h3>

---
---

## Salvamos los datos

In [7]:
df_transformar.to_csv('../../datos/datos_modelo1/tratados/escalados_propiedades_provincia_madrid.csv')